In [1]:
import pymongo
client = pymongo.MongoClient(host='120.79.139.239', port=27017)
db = client['weibo']
collection = db['users']

In [2]:
results = collection.find({}, {'id': 1})
id_list = []
for result in results:
    id_list.append(str(result['id']))
print (len(id_list))

2095


In [12]:
collection = db['weibos']
results = collection.find({}, {'created_at': 1})
time_list = []
import re
for result in results:
    if re.search('2018-07-20', result['created_at']) != None:
        time_list.append(str(result['created_at']))
time_list = sorted(time_list, reverse=True)
print (len(time_list))
print (time_list)

496
['2018-07-20 21:02', '2018-07-20 21:02', '2018-07-20 21:00', '2018-07-20 20:56', '2018-07-20 20:55', '2018-07-20 20:54', '2018-07-20 20:52', '2018-07-20 20:51', '2018-07-20 20:50', '2018-07-20 20:46', '2018-07-20 20:42', '2018-07-20 20:41', '2018-07-20 20:40', '2018-07-20 20:36', '2018-07-20 20:36', '2018-07-20 20:35', '2018-07-20 20:33', '2018-07-20 20:33', '2018-07-20 20:26', '2018-07-20 20:24', '2018-07-20 20:18', '2018-07-20 20:14', '2018-07-20 20:13', '2018-07-20 20:09', '2018-07-20 20:08', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 20:02', '2018-07-20 19:02', '2018-07-20 19:02', '2018-07-20 19:02', '2018-07-20 19:02', '2018-07-20 19:02', '2018-07-20 19:02', '2018-07-20 19:01', '2018-07-20 19:01', '2018-07-20 19:01', '2018-07-20 19:01', '2018-07-20 19:

In [39]:
import time
date = time.strftime('%Y-%m-%d', time.localtime())
ldate = time.strftime('%Y-%m-%d', time.localtime(time.time() - 24 * 60 * 60))
print (date)
print (ldate)
h = "{}|{}".format(date, ldate)
print (h)
results = collection.find({'created_date': {'$regex': h}})

2018-07-20
2018-07-19
2018-07-20|2018-07-19


In [44]:
print (results.count())

6063


/home/impulse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.
